In [ ]:
%matplotlib widget

In [ ]:
from pathlib import Path
import numpy as np
import flammkuchen as fl
import pandas as pd

import skimage as sk
import json

from split_dataset import SplitDataset
from bouterin.plots.stimulus_log_plot import get_paint_function
from scipy.cluster.hierarchy import dendrogram, linkage, cut_tree, to_tree, set_link_color_palette
from sklearn.cluster import AgglomerativeClustering
from fimpylab.core.lightsheet_experiment import LightsheetExperiment

from bouter.utilities import reliability 
from skimage.filters import threshold_otsu
import xarray as xr

from matplotlib import  pyplot as plt
import seaborn as sns
from tqdm import tqdm
sns.set(style="ticks", palette="deep")
cols = sns.color_palette()
import ipywidgets as widgets

from lotr.pca import pca_and_phase, fictive_heading_and_fit, fit_phase_neurons
from circle_fit import hyper_fit
from lotr import LotrExperiment, A_FISH

import lotr.plotting as pltltr
COLS = pltltr.COLS

from lotr.utils import roll_columns_jit

In [ ]:
def nan_phase_jumps(phase_array):
    out_array = phase_array.copy()
    out_array[1:][np.abs(np.diff(out_array)) > np.pi] = np.nan
    return out_array

In [ ]:
master =  Path(r"\\funes\Shared\experiments\E0040_motions_cardinal\v13_cw_ccw\ls_fixed\spont_plus_v13\huc")
master =  Path(r"Z:\Hagar\E0040\2p ablations\E0040V13")

fish_list = list(master.glob("*_f*"))
path = fish_list[1]
print(path)

In [ ]:
suite2p_data = fl.load(path / "data_from_suite2p_cells.h5")
coords = suite2p_data['coords']
anatomy = suite2p_data['anatomy_stack']

df = fl.load(path / "bouts_df.h5")
exp = LotrExperiment(path)
fs = int(exp.fn)
beh_df = exp.behavior_log

In [ ]:
traces = fl.load(path / "filtered_traces.h5", "/detr")

In [ ]:
suite2p_brain = fl.load(path / "data_from_suite2p_cells_brain.h5")
in_brain_idx = suite2p_brain['coords_idx']

traces = traces[:, in_brain_idx]
coords = coords[in_brain_idx]

In [ ]:
np.shape(in_brain_idx)

In [ ]:
t = np.arange(np.shape(traces)[1]) / fs

len_rec, num_traces = np.shape(traces)
print("num_traces: ", num_traces)
print("len_rec: ", len_rec)
print("sampling rate: ", fs)

In [ ]:
######################### Part 2 - looking for neurons that reliably respond to the visual stimulus
# selectnig reliable neruons 
n_blocks = 3
norm_traces = traces.T
stim_traces = norm_traces[:, ((len_rec // 2)):]
new_len_rec = len_rec // (2 * n_blocks)
print(np.shape(stim_traces))

trial_traces = np.zeros((n_blocks, num_traces, new_len_rec))
trial_traces = np.zeros((num_traces, n_blocks, new_len_rec))
trial_traces_corrected = np.zeros((num_traces, n_blocks, new_len_rec))

for i in range(n_blocks):
    t1 = i * new_len_rec
    t2 = t1 + new_len_rec
    trial_traces[:, i] = stim_traces[:, t1:t2]

In [ ]:
dt = 1 / fs
traces_xr = xr.DataArray(
    data=trial_traces,                               #Adding the data
    dims=['roi', 'block', 't'],                #Defining name of the dimensions
    coords={                                   #Defining values at which each dimension wase valuated
        'roi':np.arange(trial_traces.shape[0]), 
        'block':np.arange(n_blocks),
        't':np.arange(trial_traces.shape[2])*dt
        }
    )
reliability_arr = reliability(np.swapaxes(traces_xr, 0, 2).values)
rel_thresh = threshold_otsu(reliability_arr)
print("Reliability threshold: ", rel_thresh)

rel_thresh_3 = np.round(rel_thresh * 1000)
rel_thresh_3 /=1000



In [ ]:
rel_thresh_3 = 0.4

In [ ]:
# figure 2 - visual responses
xlim1 = 0
xlim2 = len_rec
fig_vis, ax_vis = plt.subplots(2, 2, figsize=(8, 4), gridspec_kw={'width_ratios': [1, 2], 'height_ratios': [1, 6]}) 

extent_new_ls = (0, exp.plane_ext_um[3], 0, exp.plane_ext_um[1])

exp = LotrExperiment(path)
selected_vis = np.where(reliability_arr > rel_thresh_3)[0]
#print(selected_vis)

coords_vis = coords[selected_vis, 1:] * 0.6

############ Anatomy plot of Visually responsive neurons:

for i in range(2):
    for j in range(2):
        ax_vis[i,j].axis('off')
    
anatomy_layer = np.sum(anatomy, axis=0)
anatomy_layer = np.rot90(anatomy_layer, k=2, axes=(1, 0))
ax_vis[1, 0].imshow(anatomy_layer[:, ::-1], extent=extent_new_ls, origin="upper", cmap='gray_r')#, vmin=9, vmax=100)
ax_vis[1, 0].scatter(coords_vis[:,1], coords_vis[:,0], s=5, c='purple')
pltltr.add_anatomy_scalebar(ax_vis[1, 0], pos=(-10,-10))

############ Visual stimulus:
ax_vis[0, 1].set_xlim(xlim1, xlim2)
ax_vis[0, 0].set_title("Reliability thresh: " + str(rel_thresh_3))

stimulus_log = exp.stimulus_log
stim_value, t_values = get_paint_function(stimulus_log, 'E0040_motions_cardinal')
stim_value = stim_value / 255
num_stim = np.shape(stim_value)[0]

t_values *= fs
for i in range(num_stim):
    ax_vis[0, 1].axvspan(
        t_values[i, 0],
        t_values[i, 1],
        facecolor=[
            stim_value[i, 0],
            stim_value[i, 1],
            stim_value[i, 2],
        ],
        alpha=0.7,
    )
    
############ Traces (only visually responsive taces):
tmp_traces = traces.T
rel_ind = np.where(reliability_arr > rel_thresh_3)[0]
vis_traces = tmp_traces[rel_ind, :]
im = ax_vis[1,1].imshow(
    vis_traces,
    extent=[0, exp.time_arr[-1], -np.pi, np.pi],
    aspect="auto",
    cmap=COLS["dff_plot"],
    vmin=-1.7,
    vmax=2.0,
)

pltltr.add_dff_cbar(
    im,
    ax_vis[1,1],
    (1.07, 0.04, 0.03, 0.5),
    title="ΔF (Z.)",
    titlesize=6,
    labelsize=5,
    ticklabels=None,
)

In [ ]:
file_name = "visually responsive neurons (thresh " + str(rel_thresh_3) + ").pdf"
fig_vis.savefig(path / file_name, dpi=300)

In [ ]:
d = {
    'reliability_arr': reliability_arr,
}
fl.save(path / 'reliable_rois.h5', d)